<a href="https://colab.research.google.com/github/Vaibhav-sa30/Natural-Language-Processing-using-only-Sklearn-Library/blob/main/Movie_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install squarify
!pip install contractions
!pip install opendatasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 4.6 MB/s eta 0:00:00


In [34]:
import opendatasets as od
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import contractions
import warnings
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
warnings.filterwarnings("ignore")


In [3]:
dataset_url = 'https://www.kaggle.com/competitions/sentiment-prediction-on-movie-reviews-term-break1'

In [4]:
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: vaibhav21f3002068
Your Kaggle Key: ··········


100%|██████████| 20.8M/20.8M [00:00<00:00, 57.9MB/s]



Extracting archive ./sentiment-prediction-on-movie-reviews-term-break1/sentiment-prediction-on-movie-reviews-term-break1.zip to ./sentiment-prediction-on-movie-reviews-term-break1


In [5]:
data_dir = 'sentiment-prediction-on-movie-reviews-term-break1'

In [6]:
!ls -lh {data_dir}

total 812K
drwxr-xr-x 2 root root 4.0K Sep 24 00:21 'movies.csv (1)'
-rw-r--r-- 1 root root 800K Sep 24 00:21 'sample (1).csv'
drwxr-xr-x 2 root root 4.0K Sep 24 00:21 'test.csv (1)'
drwxr-xr-x 2 root root 4.0K Sep 24 00:21 'train.csv (1)'


#Load The Data

In [130]:
train = pd.read_csv('/content/sentiment-prediction-on-movie-reviews-term-break1/train.csv (1)/train.csv')
movies = pd.read_csv('/content/sentiment-prediction-on-movie-reviews-term-break1/movies.csv (1)/movies.csv')
test = pd.read_csv('/content/sentiment-prediction-on-movie-reviews-term-break1/test.csv (1)/test.csv')

#Drop the duplicates first time

In [131]:
train = train.drop_duplicates(subset=['movieid'])
movies = movies.drop_duplicates(subset=['movieid'])
test = test.drop_duplicates(subset=['movieid'])

##Fill Null Values in `train` and `test` data

In [132]:
train['reviewText'].fillna('', inplace = True)
test['reviewText'].fillna('', inplace = True)

##Merge the `movies` and `train` data

In [133]:
train_merged = pd.merge(train, movies, on = 'movieid', how = 'left')
test_merged = pd.merge(test, movies, on = 'movieid', how = 'left')

##Drop the duplicates again!

In [134]:
train_merged = train_merged.drop_duplicates(subset = ['movieid'])
test_merged = test_merged.drop_duplicates(subset = ['movieid'])

##Fix the datatype of the column `boxOffice`

In [135]:

train_merged['boxOffice'] = train_merged['boxOffice'].str.replace('[\$,]', '',
                                                                  regex=True).str.replace('K', 'e3').str.replace('M', 'e6').astype(float)


##Segregate numerical and catagorical columns of `train_merged` data

In [136]:
train_cat = train_merged.select_dtypes(include=['object']).columns
train_num = train_merged.select_dtypes(include=['number']).columns

##Imputation for `train_merged` data

In [137]:
cat_imputer = SimpleImputer(strategy='most_frequent')
num_imputer = SimpleImputer(strategy='mean')

train_merged[train_cat] = cat_imputer.fit_transform(train_merged[train_cat])
train_merged[train_num] = num_imputer.fit_transform(train_merged[train_num])

##Do The Same For Test Data


Test data do not consist of any numerical columns.

In [138]:

test_cat = test.select_dtypes(include=['object']).columns

cat_imputer = SimpleImputer(strategy='most_frequent')

test[test_cat] = cat_imputer.fit_transform(test[test_cat])

##Define a Text Preprocessing Function

In [139]:

def expand_contractions(text):
    # Implement your own function to expand contractions here
    # Example implementation using a contraction mapping dictionary:
    contraction_mapping = {
        "ain't": "am not",
        "aren't": "are not",
        "can't": "cannot",
        "could've": "could have",
        "aren’t" : "are not" ,
        "there’s": "there is" ,
        "can’t" : "can not" ,
        "they’d" : "they had",
        "couldn’t" : "could not" ,
        "they’ll" : "they will",
        "didn’t" : "did not" ,
        "they’re" : "they are",
        "doesn’t" : "does not" ,
        "they’ve" : "they have",
        "don’t" : "do not" ,
        "we’d" : "we had",
        "hadn’t" : "had not" ,
        "we’re" : "we are",
        "hasn’t" : "has not" ,
        "we’ve" : "we have",
        "haven’t" : "have not" ,
        "weren’t" : "were not",
        "he’d" : "he had",
        "what’ll" : "what will",
        "he’ll" : "he will",
        "what’re" : "what are",
        "he’s" : "he is",
        "what’s" : "what is",
        "I’d" : "I had",
        "what’ve" : "what have",
        "I’ll" : "I will",
        "where’s" : "where is",
        "I’m" : "I am" ,
        "who’d" : "who had",
        "I’ve" : "I have" ,
        "who’ll" : "who will",
        "isn’t" : "is not" ,
        "who’re" : "who are",
        "let’s" : "let us" ,
        "who’s" : "who is",
        "mightn’t" : "might not" ,
        "who’ve" : "who have",
        "mustn’t" : "must not" ,
        "won’t" : "will not",
        "shan’t" : "shall not" ,
        "wouldn’t" : "would not",
        "she’d" : "she had",
        "you’d" : "you had",
        "she’ll" : "she will",
        "you’ll" : "you will",
        "she’s" : "she is",
        "you’re" : "you are",
        "shouldn’t" : "should not",
        "you’ve" : "you have",
        "that’s" : "that is",
    }
    for contraction, expansion in contraction_mapping.items():
        text = text.replace(contraction, expansion)
    return text

def eliminate_apostrophes(text):
    text = text.replace("'", "")
    return text

def eliminate_numbers(text):
    text = re.sub(r"\d+", "", text)
    return text

def eliminate_special_characters(text):
    special_characters = string.punctuation.replace("'", "")
    text = re.sub(rf"[{re.escape(special_characters)}]", "", text)
    return text

def remove_stopwords(text):
    stop_words = set(ENGLISH_STOP_WORDS)
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    text = " ".join(filtered_words)
    return text

#preprocessing function

def preprocess_text(text):
    text = expand_contractions(text)
    text = eliminate_apostrophes(text)
    text = eliminate_numbers(text)
    text = eliminate_special_characters(text)
    text = text.lower()

    # Step 6: Tokenization (optional, depending on subsequent steps)
    # Skip tokenization for now

    # Step 7: Stopword Removal
    text = remove_stopwords(text)

    return text

##Apply it to both `train_merged` and `test_merged` data

In [140]:
cleaned_train = train_merged.copy()
cleaned_train['reviewText'] = train_merged['reviewText'].apply(preprocess_text)

cleaned_test = test.copy()
cleaned_test['reviewText'] = test_merged['reviewText'].apply(preprocess_text)

In [94]:
cleaned_train.head()

,movieid,reviewerName,isFrequentReviewer,reviewText,sentiment,title,audienceScore,rating,ratingContents,releaseDateTheaters,releaseDateStreaming,runtimeMinutes,genre,originalLanguage,director,boxOffice,distributor,soundType
0,marvelous_pirate,Benjamin Henry,False,henry selick’s movie ’s coraline fifth stopmot...,POSITIVE,Marvelous Pirate,65.000000,PG-13,"['Violence', 'Brief Strong Language', 'Substan...",2022-10-21,2022-10-28,105.0,"Comedy, Animation, Adventure, Fantasy",English,Bennie Basso,2.533425e+07,Netflix,Surround
1,tony_montana_frodo_baggins_v_rocky_balboa,Felicia Lopez,False,cast reads like vogue oscar party guest list v...,NEGATIVE,Tony Montana Frodo Baggins V Rocky Balboa,63.180028,R,['Language'],2018-09-14,2013-01-08,91.0,Action,English,Alma White,2.533425e+07,Paramount Pictures,Surround
2,darth_vader_katniss_everdeen_sorcerer_donnie_d...,Mr. Charles Burgess,True,creed ii does slightly superior rocky sequel w...,POSITIVE,Darth Vader Katniss Everdeen Sorcerer Donnie D...,80.000000,PG-13,"['Language', 'A Scene of Sensuality', 'Sports ...",2018-11-21,2019-02-12,130.0,Drama,English,Bobby Oleary,1.157000e+08,"Warner Bros. Pictures, Metro-Goldwyn-Mayer",Surround
3,lara_croft_glimmer,Ryan Barrett,False,know youre thinking limitless bradley cooper c...,POSITIVE,Lara Croft Glimmer,63.180028,R,['Language'],2018-09-14,2020-06-02,15.0,Drama,Spanish,Eric Hutcherson,2.533425e+07,Paramount Pictures,Surround
4,jason_bourne_surreal_the_terminator_indiana_jones,Alexander Glover,False,director fernando meirelles tells story urgenc...,POSITIVE,Jason Bourne Surreal The Terminator Indiana Jones,97.000000,R,"['Language', 'Drug Content', 'Sexuality', 'Str...",2003-01-17,2004-06-08,131.0,"Crime, Drama",Portuguese (Brazil),John Trent,7.600000e+06,Miramax Films,"Surround, Dolby SR, Dolby Digital, Dolby A, Do..."


In [141]:
cleaned_train.describe()

,audienceScore,runtimeMinutes,boxOffice
count,16812.000000,16812.000000,1.681200e+04
mean,63.180028,101.930268,2.533425e+07
std,19.475450,21.156548,4.025313e+07
min,0.000000,4.000000,0.000000e+00
25%,50.000000,90.000000,2.400000e+06
50%,63.180028,100.000000,2.533425e+07
75%,78.000000,110.000000,2.533425e+07
max,100.000000,561.000000,8.584000e+08


In [142]:
cleaned_train.shape

(16812, 18)

In [144]:

num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', RobustScaler)
])

cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])


num_columns = ['audienceScore', 'runtimeMinutes', 'boxOffice']  # Replace with your numerical columns
cat_columns = ['reviewerName', 'isFrequentReviewer', 'title',
               'genre', 'originalLanguage', 'director', 'distributor',
               'soundType', 'isFrequentReviewer', 'sentiment', 'rating']

# Create a ColumnTransformer to apply different transformers to different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_columns),
        ('cat', cat_transformer, cat_columns)
    ],
    remainder='passthrough'  # Include other columns as-is
)

train_transformed = preprocessor.fit_transform(cleaned_train)


In [150]:
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,  # Maximum number of features (words) to include in the TF-IDF matrix
    stop_words='english',  # Remove common English stop words
    lowercase=True,  # Convert all text to lowercase
    ngram_range=(1, 2),  # Consider both unigrams and bigrams
)

reviewText_matrix = tfidf_vectorizer.fit_transform(cleaned_train['reviewText'])
reviewText_array = reviewText_matrix.toarray()

In [152]:
X_train, X_val, y_train, y_val = train_test_split(reviewText_array, test_size=0.2, random_state=42)

ValueError: ignored

In [148]:
train_transformed = pd.DataFrame(train_transformed, columns=cleaned_train.columns)

train_transformed['reviewText'] = ['reviewText']

train_transformed = train_transformed.reset_index().drop(['index'], axis=1)

ValueError: ignored

##Preprocess the Test Data too

In [112]:
#train_transformed = train_transformed[:, 1:]

In [116]:
cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder())
])

test_col = ['movieid', 'reviewerName', 'isTopCritic', 'reviewText']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', cat_transformer, test_col)
    ],
    remainder='passthrough'
)


test_transformed = preprocessor.fit_transform(cleaned_test)


###Features and Target Columns

In [124]:
train_transformed_df = pd.DataFrame(train_transformed)

desired_columns_order = cleaned_train.columns.tolist()

train_transformed_df = train_transformed_df[:, [train_transformed_df.columns.get_loc(col) for col in desired_columns_order]]


KeyError: ignored

KeyError: ignored

In [105]:
X = cleaned_train.drop(['sentiment'], axis=1)
y = cleaned_train['sentiment']


In [ ]:
X = X_transformed_df.drop(['sentiment'], axis = 1)
y = X_transformed_df['sentiment']

###Train and Validation Data Split (we already have test data)

In [56]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

##Model Building

In [57]:
from sklearn.svm import LinearSVC

model = LinearSVC()

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])


X_train_transformed = pipeline.fit_transform(X_train, y_train)


ValueError: ignored

In [63]:
X_test, y_test = train_test_split(cleaned_test, test_size=0.2, random_state=42)
# Transform your test data using the same pipeline
X_test_transformed = pipeline.transform(X_test)

AttributeError: ignored